# This file creates a face folder from frames folder using mediapipe face mesh model

In [19]:
import os
import numpy as np
from tqdm import tqdm
import mediapipe as mp

np.set_printoptions(suppress=True)

In [20]:
VIDEO_DIR='Videos/'
FRAMES_DIR='./Frames/' #create this folder
FACES_DIR='./Faces/' #create this folder

In [21]:
#face detection solution from Mediapipe package
mp_face_mesh = mp.solutions.face_mesh
face_mesh=mp_face_mesh.FaceMesh(max_num_faces=1,refine_landmarks=True,min_detection_confidence=0.5,min_tracking_confidence=0.5)

I0000 00:00:1731518373.806448  295567 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


In [22]:
#crops faces from the frames
import numpy as np
import cv2
from skimage import transform as trans
def preprocess(img, bbox=None, landmark=None, **kwargs):
    M = None
    image_size = [224,224]
    src = np.array([
        [30.2946, 51.6963],
        [65.5318, 51.5014],
        [48.0252, 71.7366],
        [33.5493, 92.3655],
        [62.7299, 92.2041] ], dtype=np.float32 )
    if image_size[1]==224:
        src[:,0] += 8.0
    src*=2
    if landmark is not None:
        dst = landmark.astype(np.float32)

        tform = trans.SimilarityTransform()
        tform.estimate(dst, src)
        M = tform.params[0:2,:]

    if M is None:
        if bbox is None: #use center crop
            det = np.zeros(4, dtype=np.int32)
            det[0] = int(img.shape[1]*0.0625)
            det[1] = int(img.shape[0]*0.0625)
            det[2] = img.shape[1] - det[0]
            det[3] = img.shape[0] - det[1]
        else:
            det = bbox
        margin = kwargs.get('margin', 44)
        bb = np.zeros(4, dtype=np.int32)
        bb[0] = np.maximum(det[0]-margin//2, 0)
        bb[1] = np.maximum(det[1]-margin//2, 0)
        bb[2] = np.minimum(det[2]+margin//2, img.shape[1])
        bb[3] = np.minimum(det[3]+margin//2, img.shape[0])
        ret = img[bb[1]:bb[3],bb[0]:bb[2],:]
        if len(image_size)>0:
            ret = cv2.resize(ret, (image_size[1], image_size[0]))
        return ret
    else: #do align using landmark
        assert len(image_size)==2
        warped = cv2.warpAffine(img,M,(image_size[1],image_size[0]), borderValue = 0.0)
        return warped

In [23]:
# same but uses a different model to detect face...
if not os.path.exists(FACES_DIR):
    os.makedirs(FACES_DIR)
for folder in tqdm(os.listdir(FRAMES_DIR)):
    folder_full_path=os.path.join(FACES_DIR, folder)
    if not os.path.exists(folder_full_path):
        os.makedirs(folder_full_path)

        for image_file in os.listdir(os.path.join(FRAMES_DIR, folder)):
            filename = os.path.join(FRAMES_DIR, folder, image_file)
            image = cv2.imread(filename)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            results = face_mesh.process(image_rgb)
            if results.multi_face_landmarks:
                height,width,_=image.shape
                for face_landmarks in results.multi_face_landmarks:
                    x1 = y1 = 1
                    x2 = y2 = 0
                    for id, lm in enumerate(face_landmarks.landmark):
                        cx, cy = lm.x, lm.y
                        if cx<x1:
                            x1=cx
                        if cy<y1:
                            y1=cy
                        if cx>x2:
                            x2=cx
                        if cy>y2:
                            y2=cy
                    if x1<0:
                        x1=0
                    if y1<0:
                        y1=0
                    x1,x2=int(x1*width),int(x2*width)
                    y1,y2=int(y1*height),int(y2*height)
                    face_img=image_rgb[y1:y2,x1:x2,:]
                    if np.prod(face_img.shape)==0:
                        print('Empty face ', x1,x2,y1,y2,folder,image_file)
                        continue
                    cv2.imwrite(os.path.join(folder_full_path, image_file), cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR))

100%|██████████| 582/582 [03:42<00:00,  2.61it/s]


In [9]:
#This code is designed to map video files in a directory to their corresponding number of frames, and then check whether the extracted frames stored in a directory are consistent with the total number of frames in the video.
video2len={}
for filename in sorted(os.listdir(VIDEO_DIR)):
    fn, ext = os.path.splitext(os.path.basename(filename))
    vid=os.path.join(VIDEO_DIR,filename)
    cap = cv2.VideoCapture(vid)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video2len[fn]=total_frames #+1 #FIX ME!!! NEED TO ADD 1 to teh number of frames for consistency with challeng's organizer

    dn=os.path.join(FRAMES_DIR,fn)
    last_img=''
    if os.path.exists(dn):
        images=[img_name for img_name in os.listdir(dn) if img_name.lower().endswith('.png')]
        last_img=sorted(images, key=compare_filenames)[-1]
    if True or str(total_frames) not in last_img:
        print(fn,total_frames,last_img)

.DS_Store 0 
Bqb2wT_eP_4 0 
Uh00UIl7-bo 0 
Ul53TVUR4NM 0 
UpFG5ZK62XM 0 
VKg82m-RaQM 0 
Zwf-fZdgtYA 0 
aJKL0ahn1Dk 0 
ngITkMvWuq8 0 
uLpLviZ4Hc4 0 


OpenCV: Couldn't read video stream from file "Videos/.DS_Store"
OpenCV: Couldn't read video stream from file "Videos/Bqb2wT_eP_4"
OpenCV: Couldn't read video stream from file "Videos/Uh00UIl7-bo"
OpenCV: Couldn't read video stream from file "Videos/Ul53TVUR4NM"
OpenCV: Couldn't read video stream from file "Videos/UpFG5ZK62XM"
OpenCV: Couldn't read video stream from file "Videos/VKg82m-RaQM"
OpenCV: Couldn't read video stream from file "Videos/Zwf-fZdgtYA"
OpenCV: Couldn't read video stream from file "Videos/aJKL0ahn1Dk"
OpenCV: Couldn't read video stream from file "Videos/ngITkMvWuq8"
OpenCV: Couldn't read video stream from file "Videos/uLpLviZ4Hc4"


In [11]:
#compare images in frames folder and images in faces folder
for folder in os.listdir(FRAMES_DIR):
    folder_full_path=os.path.join(FACES_DIR, folder)
    print(folder_full_path)
    if not os.path.exists(folder_full_path):
        print('No faces for ',folder)
        continue
    for image_file in os.listdir(os.path.join(FRAMES_DIR, folder)):
        filename = os.path.join(FRAMES_DIR, folder, image_file)
        face_filename = os.path.join(FACES_DIR, folder, image_file)
        if not os.path.exists(face_filename):
            print('No face for ',filename)

./Faces/Ul53TVUR4NM_8_0_34907
./Faces/Ul53TVUR4NM_7_6_24509
No face for  ./Frames/Ul53TVUR4NM_7_6_24509/24509.jpg
./Faces/Ul53TVUR4NM_8_0_34799
No face for  ./Frames/Ul53TVUR4NM_8_0_34799/34877.jpg
./Faces/Ul53TVUR4NM_11_4_74082
No face for  ./Frames/Ul53TVUR4NM_11_4_74082/74088.jpg
No face for  ./Frames/Ul53TVUR4NM_11_4_74082/74112.jpg
No face for  ./Frames/Ul53TVUR4NM_11_4_74082/74082.jpg
No face for  ./Frames/Ul53TVUR4NM_11_4_74082/74124.jpg
./Faces/Uh00UIl7-bo_1_13_14887
./Faces/Uh00UIl7-bo_1_2_8827
No face for  ./Frames/Uh00UIl7-bo_1_2_8827/9121.jpg
No face for  ./Frames/Uh00UIl7-bo_1_2_8827/9109.jpg
No face for  ./Frames/Uh00UIl7-bo_1_2_8827/9133.jpg
No face for  ./Frames/Uh00UIl7-bo_1_2_8827/9127.jpg
No face for  ./Frames/Uh00UIl7-bo_1_2_8827/9145.jpg
No face for  ./Frames/Uh00UIl7-bo_1_2_8827/8863.jpg
No face for  ./Frames/Uh00UIl7-bo_1_2_8827/9115.jpg
No face for  ./Frames/Uh00UIl7-bo_1_2_8827/9103.jpg
No face for  ./Frames/Uh00UIl7-bo_1_2_8827/8839.jpg
./Faces/UpFG5ZK62XM_4_4

In [12]:
len(os.listdir(FRAMES_DIR)),len(os.listdir(FACES_DIR))

(582, 583)

In [14]:
#print image count in faces folder
for folder in os.listdir(FACES_DIR):
    if folder=='.DS_Store':
        continue
    print(folder,len(os.listdir(os.path.join(FACES_DIR, folder))))
    print(folder,len(os.listdir(os.path.join(FRAMES_DIR, folder))))

Ul53TVUR4NM_8_0_34907 18
Ul53TVUR4NM_8_0_34907 18
Ul53TVUR4NM_7_6_24509 17
Ul53TVUR4NM_7_6_24509 18
Ul53TVUR4NM_8_0_34799 17
Ul53TVUR4NM_8_0_34799 18
Ul53TVUR4NM_11_4_74082 4
Ul53TVUR4NM_11_4_74082 8
Uh00UIl7-bo_1_13_14887 6
Uh00UIl7-bo_1_13_14887 6
Uh00UIl7-bo_1_2_8827 9
Uh00UIl7-bo_1_2_8827 18
UpFG5ZK62XM_4_4_69060 4
UpFG5ZK62XM_4_4_69060 11
Ul53TVUR4NM_9_2_35160 10
Ul53TVUR4NM_9_2_35160 18
Zwf-fZdgtYA_1_4_4465 16
Zwf-fZdgtYA_1_4_4465 18
ngITkMvWuq8_2_0_20011 0
ngITkMvWuq8_2_0_20011 2
Uh00UIl7-bo_1_6_15511 12
Uh00UIl7-bo_1_6_15511 18
Ul53TVUR4NM_7_1_24743 16
Ul53TVUR4NM_7_1_24743 18
Ul53TVUR4NM_9_0_35556 0
Ul53TVUR4NM_9_0_35556 18
Uh00UIl7-bo_1_4_11119 17
Uh00UIl7-bo_1_4_11119 18
Uh00UIl7-bo_1_5_13975 12
Uh00UIl7-bo_1_5_13975 18
UpFG5ZK62XM_4_2_68898 8
UpFG5ZK62XM_4_2_68898 9
aJKL0ahn1Dk_1_1_20114 0
aJKL0ahn1Dk_1_1_20114 9
Ul53TVUR4NM_5_0_16308 18
Ul53TVUR4NM_5_0_16308 18
Zwf-fZdgtYA_1_0_4291 0
Zwf-fZdgtYA_1_0_4291 6
ngITkMvWuq8_3_7_21691 16
ngITkMvWuq8_3_7_21691 18
Uh00UIl7-bo_1_9_1

In [17]:
count=0
for folder in os.listdir(FACES_DIR):
    if folder=='.DS_Store':
        continue
    len_faces = len(os.listdir(os.path.join(FACES_DIR, folder)))
    len_frames = len(os.listdir(os.path.join(FRAMES_DIR, folder)))
    
    if len_faces != len_frames:
        print(folder, len_faces, len_frames)
    if len_faces == 0:
        count+=1
        print(folder)

Ul53TVUR4NM_7_6_24509 17 18
Ul53TVUR4NM_8_0_34799 17 18
Ul53TVUR4NM_11_4_74082 4 8
Uh00UIl7-bo_1_2_8827 9 18
UpFG5ZK62XM_4_4_69060 4 11
Ul53TVUR4NM_9_2_35160 10 18
Zwf-fZdgtYA_1_4_4465 16 18
ngITkMvWuq8_2_0_20011 0 2
ngITkMvWuq8_2_0_20011
Uh00UIl7-bo_1_6_15511 12 18
Ul53TVUR4NM_7_1_24743 16 18
Ul53TVUR4NM_9_0_35556 0 18
Ul53TVUR4NM_9_0_35556
Uh00UIl7-bo_1_4_11119 17 18
Uh00UIl7-bo_1_5_13975 12 18
UpFG5ZK62XM_4_2_68898 8 9
aJKL0ahn1Dk_1_1_20114 0 9
aJKL0ahn1Dk_1_1_20114
Zwf-fZdgtYA_1_0_4291 0 6
Zwf-fZdgtYA_1_0_4291
ngITkMvWuq8_3_7_21691 16 18
Uh00UIl7-bo_1_9_15601 12 18
Uh00UIl7-bo_1_2_8077 12 18
Uh00UIl7-bo_1_2_11635 5 6
aJKL0ahn1Dk_1_0_19532 17 18
UpFG5ZK62XM_4_1_68904 15 16
UpFG5ZK62XM_4_0_69726 0 6
UpFG5ZK62XM_4_0_69726
Ul53TVUR4NM_9_1_35268 12 18
ngITkMvWuq8_3_1_22159 12 13
UpFG5ZK62XM_2_1_68040 1 2
Ul53TVUR4NM_2_2_4831 4 6
Ul53TVUR4NM_10_2_63655 15 18
Ul53TVUR4NM_3_11_12163 2 3
ngITkMvWuq8_3_7_21433 16 17
uLpLviZ4Hc4_1_0_5677 5 6
Ul53TVUR4NM_1_1_3127 6 18
Uh00UIl7-bo_1_0_11431 8 1

In [18]:
print(
    len(os.listdir(FACES_DIR)),
    len(os.listdir(FRAMES_DIR)),
    count
)

583 582 47
